In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

Data Generation
===

In [2]:
np.random.seed(10)
p, q = (np.random.rand(i, 2) for i in (4, 5))
p_big, q_big = (np.random.rand(i, 80) for i in (100, 120))

print(p, "\n\n", q)

[[ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]] 

 [[ 0.16911084  0.08833981]
 [ 0.68535982  0.95339335]
 [ 0.00394827  0.51219226]
 [ 0.81262096  0.61252607]
 [ 0.72175532  0.29187607]]


Solution
===

In [3]:
def naive(p, q):
    ''' fill your code in here...
    '''
    x = np.zeros((len(p),len(q)))
    print(x)
    
    for a in range(0,len(p)):
        for b in range(0,len(q)):
            print(str(a) + " " + str(b))
            for i in range (0,p.shape[1]):
                x[a,b] += (p[a,i] - q[b,i])**2
            x[a,b] = np.sqrt(x[a,b])
    return x
print(naive(p,q))

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
[[ 0.60599073  0.93659449  0.91124856  0.59321356  0.27561751]
 [ 0.80746999  0.21102354  0.67268649  0.22495084  0.46534491]
 [ 0.35654215  0.75217493  0.57200052  0.49900068  0.23310825]
 [ 0.67281411  0.52407472  0.31520226  0.63212897  0.70277376]]


### Use matching indices

Instead of iterating through indices, one can use them directly to parallelize the operations with Numpy.

In [4]:
rows, cols = np.indices((p.shape[0], q.shape[0]))
print(rows, end='\n\n')
print(cols)

[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]]

[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]


In [5]:
print(p[rows.ravel()], end='\n\n')
print(q[cols.ravel()])

[[ 0.77132064  0.02075195]
 [ 0.77132064  0.02075195]
 [ 0.77132064  0.02075195]
 [ 0.77132064  0.02075195]
 [ 0.77132064  0.02075195]
 [ 0.63364823  0.74880388]
 [ 0.63364823  0.74880388]
 [ 0.63364823  0.74880388]
 [ 0.63364823  0.74880388]
 [ 0.63364823  0.74880388]
 [ 0.49850701  0.22479665]
 [ 0.49850701  0.22479665]
 [ 0.49850701  0.22479665]
 [ 0.49850701  0.22479665]
 [ 0.49850701  0.22479665]
 [ 0.19806286  0.76053071]
 [ 0.19806286  0.76053071]
 [ 0.19806286  0.76053071]
 [ 0.19806286  0.76053071]
 [ 0.19806286  0.76053071]]

[[ 0.16911084  0.08833981]
 [ 0.68535982  0.95339335]
 [ 0.00394827  0.51219226]
 [ 0.81262096  0.61252607]
 [ 0.72175532  0.29187607]
 [ 0.16911084  0.08833981]
 [ 0.68535982  0.95339335]
 [ 0.00394827  0.51219226]
 [ 0.81262096  0.61252607]
 [ 0.72175532  0.29187607]
 [ 0.16911084  0.08833981]
 [ 0.68535982  0.95339335]
 [ 0.00394827  0.51219226]
 [ 0.81262096  0.61252607]
 [ 0.72175532  0.29187607]
 [ 0.16911084  0.08833981]
 [ 0.68535982  0.95339335]

In [6]:
def with_indices(p, q):
    ''' fill your code in here...
    '''
    rows, cols = np.indices((p.shape[0], q.shape[0]))
    r = np.sqrt(np.sum((p[rows.ravel(),:] - q[cols.ravel(),:])**2,axis = 1))
    r = r.reshape((4,5))
    return r
print(with_indices(p, q))

[[ 0.60599073  0.93659449  0.91124856  0.59321356  0.27561751]
 [ 0.80746999  0.21102354  0.67268649  0.22495084  0.46534491]
 [ 0.35654215  0.75217493  0.57200052  0.49900068  0.23310825]
 [ 0.67281411  0.52407472  0.31520226  0.63212897  0.70277376]]


### Use a library

`scipy` is the equivalent of matlab toolboxes and have a lot to offer. Actually the pairwise computation is part of the library through the `spatial` module.

In [7]:
from scipy.spatial.distance import cdist

def scipy_version(p, q):
    return cdist(p, q)

### Numpy Magic

In [15]:
print (p[:,np.newaxis,:])

def tensor_broadcasting(p, q):
    #print (q[np.newaxis,:,:])
    print(p[:,np.newaxis,:].shape)
    print(q[np.newaxis,:,:].shape)
    return np.sqrt(np.sum((p[:,np.newaxis,:]-q[np.newaxis,:,:])**2, axis=2))
print (tensor_broadcasting(p,q))

[[[ 0.77132064  0.02075195]]

 [[ 0.63364823  0.74880388]]

 [[ 0.49850701  0.22479665]]

 [[ 0.19806286  0.76053071]]]
(4, 1, 2)
(1, 5, 2)
[[ 0.60599073  0.93659449  0.91124856  0.59321356  0.27561751]
 [ 0.80746999  0.21102354  0.67268649  0.22495084  0.46534491]
 [ 0.35654215  0.75217493  0.57200052  0.49900068  0.23310825]
 [ 0.67281411  0.52407472  0.31520226  0.63212897  0.70277376]]


# Compare methods

In [28]:
methods = [naive, with_indices, scipy_version, tensor_broadcasting]
timers = []
for f in methods:
    r = %timeit -o f(p_big, q_big)
    timers.append(r)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39


13 82
13 83
13 84
13 85
13 86
13 87
13 88
13 89
13 90
13 91
13 92
13 93
13 94
13 95
13 96
13 97
13 98
13 99
13 100
13 101
13 102
13 103
13 104
13 105
13 106
13 107
13 108
13 109
13 110
13 111
13 112
13 113
13 114
13 115
13 116
13 117
13 118
13 119
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
14 14
14 15
14 16
14 17
14 18
14 19
14 20
14 21
14 22
14 23
14 24
14 25
14 26
14 27
14 28
14 29
14 30
14 31
14 32
14 33
14 34
14 35
14 36
14 37
14 38
14 39
14 40
14 41
14 42
14 43
14 44
14 45
14 46
14 47
14 48
14 49
14 50
14 51
14 52
14 53
14 54
14 55
14 56
14 57
14 58
14 59
14 60
14 61
14 62
14 63
14 64
14 65
14 66
14 67
14 68
14 69
14 70
14 71
14 72
14 73
14 74
14 75
14 76
14 77
14 78
14 79
14 80
14 81
14 82
14 83
14 84
14 85
14 86
14 87
14 88
14 89
14 90
14 91
14 92
14 93
14 94
14 95
14 96
14 97
14 98
14 99
14 100
14 101
14 102
14 103
14 104
14 105
14 106
14 107
14 108
14 109
14 110
14 111
14 112
14 113
14 114
14 115
14 116
14 117
14 118
14 119
15 0
15 1
15 2
15 3
15

26 20
26 21
26 22
26 23
26 24
26 25
26 26
26 27
26 28
26 29
26 30
26 31
26 32
26 33
26 34
26 35
26 36
26 37
26 38
26 39
26 40
26 41
26 42
26 43
26 44
26 45
26 46
26 47
26 48
26 49
26 50
26 51
26 52
26 53
26 54
26 55
26 56
26 57
26 58
26 59
26 60
26 61
26 62
26 63
26 64
26 65
26 66
26 67
26 68
26 69
26 70
26 71
26 72
26 73
26 74
26 75
26 76
26 77
26 78
26 79
26 80
26 81
26 82
26 83
26 84
26 85
26 86
26 87
26 88
26 89
26 90
26 91
26 92
26 93
26 94
26 95
26 96
26 97
26 98
26 99
26 100
26 101
26 102
26 103
26 104
26 105
26 106
26 107
26 108
26 109
26 110
26 111
26 112
26 113
26 114
26 115
26 116
26 117
26 118
26 119
27 0
27 1
27 2
27 3
27 4
27 5
27 6
27 7
27 8
27 9
27 10
27 11
27 12
27 13
27 14
27 15
27 16
27 17
27 18
27 19
27 20
27 21
27 22
27 23
27 24
27 25
27 26
27 27
27 28
27 29
27 30
27 31
27 32
27 33
27 34
27 35
27 36
27 37
27 38
27 39
27 40
27 41
27 42
27 43
27 44
27 45
27 46
27 47
27 48
27 49
27 50
27 51
27 52
27 53
27 54
27 55
27 56
27 57
27 58
27 59
27 60
27 61
27 62
27 63
27 64


38 78
38 79
38 80
38 81
38 82
38 83
38 84
38 85
38 86
38 87
38 88
38 89
38 90
38 91
38 92
38 93
38 94
38 95
38 96
38 97
38 98
38 99
38 100
38 101
38 102
38 103
38 104
38 105
38 106
38 107
38 108
38 109
38 110
38 111
38 112
38 113
38 114
38 115
38 116
38 117
38 118
38 119
39 0
39 1
39 2
39 3
39 4
39 5
39 6
39 7
39 8
39 9
39 10
39 11
39 12
39 13
39 14
39 15
39 16
39 17
39 18
39 19
39 20
39 21
39 22
39 23
39 24
39 25
39 26
39 27
39 28
39 29
39 30
39 31
39 32
39 33
39 34
39 35
39 36
39 37
39 38
39 39
39 40
39 41
39 42
39 43
39 44
39 45
39 46
39 47
39 48
39 49
39 50
39 51
39 52
39 53
39 54
39 55
39 56
39 57
39 58
39 59
39 60
39 61
39 62
39 63
39 64
39 65
39 66
39 67
39 68
39 69
39 70
39 71
39 72
39 73
39 74
39 75
39 76
39 77
39 78
39 79
39 80
39 81
39 82
39 83
39 84
39 85
39 86
39 87
39 88
39 89
39 90
39 91
39 92
39 93
39 94
39 95
39 96
39 97
39 98
39 99
39 100
39 101
39 102
39 103
39 104
39 105
39 106
39 107
39 108
39 109
39 110
39 111
39 112
39 113
39 114
39 115
39 116
39 117
39 118
39 11

51 13
51 14
51 15
51 16
51 17
51 18
51 19
51 20
51 21
51 22
51 23
51 24
51 25
51 26
51 27
51 28
51 29
51 30
51 31
51 32
51 33
51 34
51 35
51 36
51 37
51 38
51 39
51 40
51 41
51 42
51 43
51 44
51 45
51 46
51 47
51 48
51 49
51 50
51 51
51 52
51 53
51 54
51 55
51 56
51 57
51 58
51 59
51 60
51 61
51 62
51 63
51 64
51 65
51 66
51 67
51 68
51 69
51 70
51 71
51 72
51 73
51 74
51 75
51 76
51 77
51 78
51 79
51 80
51 81
51 82
51 83
51 84
51 85
51 86
51 87
51 88
51 89
51 90
51 91
51 92
51 93
51 94
51 95
51 96
51 97
51 98
51 99
51 100
51 101
51 102
51 103
51 104
51 105
51 106
51 107
51 108
51 109
51 110
51 111
51 112
51 113
51 114
51 115
51 116
51 117
51 118
51 119
52 0
52 1
52 2
52 3
52 4
52 5
52 6
52 7
52 8
52 9
52 10
52 11
52 12
52 13
52 14
52 15
52 16
52 17
52 18
52 19
52 20
52 21
52 22
52 23
52 24
52 25
52 26
52 27
52 28
52 29
52 30
52 31
52 32
52 33
52 34
52 35
52 36
52 37
52 38
52 39
52 40
52 41
52 42
52 43
52 44
52 45
52 46
52 47
52 48
52 49
52 50
52 51
52 52
52 53
52 54
52 55
52 56
52 57


...

33 31
33 32
33 33
33 34
33 35
33 36
33 37
33 38
33 39
33 40
33 41
33 42
33 43
33 44
33 45
33 46
33 47
33 48
33 49
33 50
33 51
33 52
33 53
33 54
33 55
33 56
33 57
33 58
33 59
33 60
33 61
33 62
33 63
33 64
33 65
33 66
33 67
33 68
33 69
33 70
33 71
33 72
33 73
33 74
33 75
33 76
33 77
33 78
33 79
33 80
33 81
33 82
33 83
33 84
33 85
33 86
33 87
33 88
33 89
33 90
33 91
33 92
33 93
33 94
33 95
33 96
33 97
33 98
33 99
33 100
33 101
33 102
33 103
33 104
33 105
33 106
33 107
33 108
33 109
33 110
33 111
33 112
33 113
33 114
33 115
33 116
33 117
33 118
33 119
34 0
34 1
34 2
34 3
34 4
34 5
34 6
34 7
34 8
34 9
34 10
34 11
34 12
34 13
34 14
34 15
34 16
34 17
34 18
34 19
34 20
34 21
34 22
34 23
34 24
34 25
34 26
34 27
34 28
34 29
34 30
34 31
34 32
34 33
34 34
34 35
34 36
34 37
34 38
34 39
34 40
34 41
34 42
34 43
34 44
34 45
34 46
34 47
34 48
34 49
34 50
34 51
34 52
34 53
34 54
34 55
34 56
34 57
34 58
34 59
34 60
34 61
34 62
34 63
34 64
34 65
34 66
34 67
34 68
34 69
34 70
34 71
34 72
34 73
34 74
34 75


50 106
50 107
50 108
50 109
50 110
50 111
50 112
50 113
50 114
50 115
50 116
50 117
50 118
50 119
51 0
51 1
51 2
51 3
51 4
51 5
51 6
51 7
51 8
51 9
51 10
51 11
51 12
51 13
51 14
51 15
51 16
51 17
51 18
51 19
51 20
51 21
51 22
51 23
51 24
51 25
51 26
51 27
51 28
51 29
51 30
51 31
51 32
51 33
51 34
51 35
51 36
51 37
51 38
51 39
51 40
51 41
51 42
51 43
51 44
51 45
51 46
51 47
51 48
51 49
51 50
51 51
51 52
51 53
51 54
51 55
51 56
51 57
51 58
51 59
51 60
51 61
51 62
51 63
51 64
51 65
51 66
51 67
51 68
51 69
51 70
51 71
51 72
51 73
51 74
51 75
51 76
51 77
51 78
51 79
51 80
51 81
51 82
51 83
51 84
51 85
51 86
51 87
51 88
51 89
51 90
51 91
51 92
51 93
51 94
51 95
51 96
51 97
51 98
51 99
51 100
51 101
51 102
51 103
51 104
51 105
51 106
51 107
51 108
51 109
51 110
51 111
51 112
51 113
51 114
51 115
51 116
51 117
51 118
51 119
52 0
52 1
52 2
52 3
52 4
52 5
52 6
52 7
52 8
52 9
52 10
52 11
52 12
52 13
52 14
52 15
52 16
52 17
52 18
52 19
52 20
52 21
52 22
52 23
52 24
52 25
52 26
52 27
52 28
52 29
52

70 98
70 99
70 100
70 101
70 102
70 103
70 104
70 105
70 106
70 107
70 108
70 109
70 110
70 111
70 112
70 113
70 114
70 115
70 116
70 117
70 118
70 119
71 0
71 1
71 2
71 3
71 4
71 5
71 6
71 7
71 8
71 9
71 10
71 11
71 12
71 13
71 14
71 15
71 16
71 17
71 18
71 19
71 20
71 21
71 22
71 23
71 24
71 25
71 26
71 27
71 28
71 29
71 30
71 31
71 32
71 33
71 34
71 35
71 36
71 37
71 38
71 39
71 40
71 41
71 42
71 43
71 44
71 45
71 46
71 47
71 48
71 49
71 50
71 51
71 52
71 53
71 54
71 55
71 56
71 57
71 58
71 59
71 60
71 61
71 62
71 63
71 64
71 65
71 66
71 67
71 68
71 69
71 70
71 71
71 72
71 73
71 74
71 75
71 76
71 77
71 78
71 79
71 80
71 81
71 82
71 83
71 84
71 85
71 86
71 87
71 88
71 89
71 90
71 91
71 92
71 93
71 94
71 95
71 96
71 97
71 98
71 99
71 100
71 101
71 102
71 103
71 104
71 105
71 106
71 107
71 108
71 109
71 110
71 111
71 112
71 113
71 114
71 115
71 116
71 117
71 118
71 119
72 0
72 1
72 2
72 3
72 4
72 5
72 6
72 7
72 8
72 9
72 10
72 11
72 12
72 13
72 14
72 15
72 16
72 17
72 18
72 19
72 20
72

87 23
87 24
87 25
87 26
87 27
87 28
87 29
87 30
87 31
87 32
87 33
87 34
87 35
87 36
87 37
87 38
87 39
87 40
87 41
87 42
87 43
87 44
87 45
87 46
87 47
87 48
87 49
87 50
87 51
87 52
87 53
87 54
87 55
87 56
87 57
87 58
87 59
87 60
87 61
87 62
87 63
87 64
87 65
87 66
87 67
87 68
87 69
87 70
87 71
87 72
87 73
87 74
87 75
87 76
87 77
87 78
87 79
87 80
87 81
87 82
87 83
87 84
87 85
87 86
87 87
87 88
87 89
87 90
87 91
87 92
87 93
87 94
87 95
87 96
87 97
87 98
87 99
87 100
87 101
87 102
87 103
87 104
87 105
87 106
87 107
87 108
87 109
87 110
87 111
87 112
87 113
87 114
87 115
87 116
87 117
87 118
87 119
88 0
88 1
88 2
88 3
88 4
88 5
88 6
88 7
88 8
88 9
88 10
88 11
88 12
88 13
88 14
88 15
88 16
88 17
88 18
88 19
88 20
88 21
88 22
88 23
88 24
88 25
88 26
88 27
88 28
88 29
88 30
88 31
88 32
88 33
88 34
88 35
88 36
88 37
88 38
88 39
88 40
88 41
88 42
88 43
88 44
88 45
88 46
88 47
88 48
88 49
88 50
88 51
88 52
88 53
88 54
88 55
88 56
88 57
88 58
88 59
88 60
88 61
88 62
88 63
88 64
88 65
88 66
88 67


ValueError: cannot reshape array of size 12000 into shape (4,5)

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(np.arange(len(methods)), [r.best*1000 for r in timers], log=False)  # Set log to True for logarithmic scale
plt.xticks(np.arange(len(methods))+0.2, [f.__name__ for f in methods], rotation=30)
plt.xlabel('Method')
plt.ylabel('Time (ms)')
plt.show()